In [1]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from datasets import ClassLabel, Value
from torchdata.nodes.adapters import IterableWrapper
from torchdata.nodes.batch import Batcher
from torchdata.nodes.map import Mapper, ParallelMapper
from torchdata.nodes.prefetch import Prefetcher
import numpy as np
dataset = load_dataset("ylecun/mnist") 
dataset = dataset.with_format("torch")
from torchdata.stateful_dataloader import StatefulDataLoader

In [2]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

train_node = IterableWrapper(train_dataset)
test_node = IterableWrapper(test_dataset)



In [3]:
train_dataset[0]

{'image': tensor([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
            18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
          [  

In [4]:
print("length of Train Datasets: ", len(train_dataset))
print("length of Validation Datasets: ", len(test_dataset))

length of Train Datasets:  60000
length of Validation Datasets:  10000


In [5]:
batch_size = 2
epochs = 100
input_size = 28 * 28
num_classes = 10

In [6]:
train_batcher = Batcher(train_node, batch_size, drop_last=True)
test_batcher = Batcher(test_node, batch_size, drop_last=True)


In [7]:
dataloader = StatefulDataLoader(train_dataset, batch_size=batch_size)

In [8]:
for batch in dataloader:
    print(type(batch))
    print(batch)
    break

<class 'dict'>
{'image': tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]]], dtype=torch.uint8), 'label': tensor([5, 0])}


In [9]:
for batch in train_batcher:
    print(type(batch))
    print(batch)
    break

<class 'list'>
[{'image': tensor([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
           18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
        